# Importing the libraries

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Model Synthesizers

In [2]:
def define_GAN(g_model, d_model):
    d_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

In [3]:
def define_discriminator(input_shape= (32,32,3)):
    model = keras.models.Sequential()
    # Input is a 32*32*3 image
    model.add(keras.layers.Conv2D(filters= 64,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Conv2D(filters= 64,
                                  kernel_size= (3,3),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # Input is a 16*16*3 image
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # Input is a 8*8*3 image
    model.add(keras.layers.Conv2D(filters= 256,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # Input is now 4*4*3
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(units= 1,
                                 activation= 'sigmoid'))
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])
    
    return model

In [4]:
def define_generator(latent_dim):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units= 256 * 4 * 4, input_dim= latent_dim))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Reshape((4, 4, 256)))
    # 8 * 8 now
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 16 * 16 now
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 32 * 32 now
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Conv2D(filters= 3,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  activation= 'tanh'))
    
    return model

# Generating points

In [5]:
def load_data():
    (X_train, _), (_, _) = keras.datasets.cifar10.load_data()
    return X_train

In [13]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape((n_samples, latent_dim))
    return X

In [12]:
def generate_real_sample(n_samples):
    data = load_data()
    ix = np.random.randint(0,len(data), n_samples)
    X = data[ix]
    X = X.reshape((n_samples, 32, 32, 3)).astype('float32')
    X = (X - 127.5) / 127.5
    y = np.ones((n_samples, 1))
    return X, y

In [11]:
def generate_fake_sample(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim= latent_dim,
                                     n_samples= n_samples)
    
    X = g_model.predict(x_input)
    y = np.ones((n_samples, 1))
    print(X.shape)
    return X, y

# Trying out the model

In [14]:
latent_dim = 100
g_model, d_model = define_generator(latent_dim), define_discriminator()
gan_model = define_GAN(g_model= g_model, d_model= d_model)
gan_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 32, 32, 3)         1466115   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 698561    
Total params: 2,164,676
Trainable params: 1,466,115
Non-trainable params: 698,561
_________________________________________________________________


# Summarizing and plotting the model

# GAN trainer

In [ ]:
def train_GAN(gan_model, g_model, d_model, dataset, latent_dim, iters= 100, batch_size= 256):
    half_batch = int(batch_size / 2)
    batch_per_epoch = int(len(dataset) / batch_size)
    for i in range(iters):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_sample(n_samples= half_batch)
            X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                                  latent_dim= latent_dim,
                                                  n_samples= half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            d_model.train_on_batch(X, y)
            
            x_input = generate_latent_points(latent_dim= latent_dim,
                                             n_samples= batch_size)
            x_gan = g_model.predict(x_input)
            y_gan = np.ones((batch_size, 1))
            gan_model.train_on_batch(x_gan, y_gan)